In [3]:
from pathlib import Path
import configparser
from psycopg2 import connect
import pandas.io.sql as pandasql
from IPython.display import HTML
from psycopg2.extras import execute_values
import requests
from psycopg2.extras import RealDictCursor
import json
CONFIG = configparser.ConfigParser()
CONFIG.read(str(Path.home().joinpath('db.cfg')))
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [668]:
# First time running
#null_matched= []
#unmatched_1 = matching(60, 0.60, 45, 1, None, 'gis_shared_streets.centreline_both_dir',  'gis_shared_streets.centreline_matched', con)
#print(unmatched_1)

# Second time running
#unmatched_2 = matching(60, 0.3, 45, 2, unmatched_1, 'gis_shared_streets.centreline_both_dir', 'gis_shared_streets.centreline_matched', con)
#len(unmatched_2)

# third time running

sql= '''create view gis_shared_streets.centreline_streets_only as (select geo_id, direction, dir_geom AS geom from gis_shared_streets.centreline_both_dir where FCODE_DESC IN ('Collector','Collector Ramp',
                'Local','Major Arterial','Major Arterial Ramp','Minor Arterial',
                'Minor Arterial Ramp','Pending') ORDER BY geo_id)'''
with con:
    with con.cursor(cursor_factory=RealDictCursor) as cur:
                    cur.execute(sql)


In [5]:
def create_query_info_table(layer_name):
    sql = 'CREATE TABLE gis_shared_streets.'+layer_name+'_query_info_table (like gis_shared_streets.platonic_ideal_query_info_table)'
    with con:
            with con.cursor() as cur:
                cur.execute(sql)

In [6]:
create_query_info_table('bluetooth')

In [7]:
unmatched_1_bt = matching(60, 0.8, 25, 1, None, 'bluetooth.segments', 'gis_shared_streets.bluetooth_matched','gis_shared_streets.bluetooth_query_info', 'analysis_id', None,  con)
len(unmatched_1_bt)

KeyboardInterrupt: 

In [1]:
def matching(search_radius, length_tolerance, bearing_tolerance, query_num, previously_unmatched_id, input_table, output_table, query_info_table, primary_key1, primary_key2, con):
    
    if previously_unmatched_id == []:
        return "Everything is matched"
    
    
    if previously_unmatched_id is None:    
        count_sql = '''SELECT COUNT(*) 
                FROM ''' + input_table
            
        with con:
            with con.cursor(cursor_factory=RealDictCursor) as cur:
                    cur.execute(count_sql)
                    count_results = cur.fetchall()
                    count = count_results[0]['count']
                                  
    else: 
        count = len(previously_unmatched_id)
            
    
    # list for all the ids that do not get matched 
    unmatched_rows = []
    
    if primary_key2 is not None:
        for i in range(0, count, 300):
        
            # first time calling function on input table         
            if previously_unmatched_id is None: 

                # two rows make up the primary key of the table
                get_centreline_sql =  '''SELECT 'Feature' as type,  json_build_object('{}', {}, '{}', {}) as properties, ST_AsGeoJson(geom)::json as geometry 
                FROM {}
                LIMIT 300 OFFSET {}'''.format(primary_key1, str(primary_key1), primary_key2, str(primary_key2), input_table, i)


            # calling function again
            else:

                get_centreline_sql = '''SELECT 'Feature' as type,  json_build_object('{}', {}, '{}', {}) as properties, ST_AsGeoJson(geom)::json as geometry 
                FROM {} WHERE {}||{} IN {} 
                LIMIT 300 OFFSET {}'''.format(primary_key1, primary_key1, primary_key2, primary_key2, input_table, str(primary_key1), str(primary_key2),tuple(previously_unmatched_id), i)

            with con:
                with con.cursor(cursor_factory=RealDictCursor) as cur:
                    cur.execute(get_centreline_sql)
                    results = cur.fetchall()
                    final_json = json.dumps({"type": "FeatureCollection",
                                            "features": results})
            # Send data to SharedStreets API    
            base_url = "https://api.sharedstreets.io/v0.1.0/match/geoms"
            apiKey = "bdd23fa1-7ac5-4158-b354-22ec946bb575"
            params = {'ignoreDirection': 'false',
                      'bearingTolerance' : bearing_tolerance,
                      'searchRadius' : search_radius,
                      'auth' : apiKey,
                      'tileHierarchy': 6,
                      'dataSource': 'osm/planet-181029',
                      'lengthTolerance' : length_tolerance,
                      'snapTopology': 'true'
                     }
            headers= {"Content-type": "application/json; charset=UTF-8"}

            r = requests.post(base_url, params = params, headers = headers, json = {"type": "FeatureCollection", "features": results})
            returned_json = r.json()
            unmatched = returned_json.get('unmatched').get('features')
            for feature in unmatched:

                    unmatched_id1 = feature['properties']['{}'.format(primary_key1)]
                    unmatched_id2 = feature['properties']['{}'.format(primary_key2)]
                    unmatched_rows.append(str(unmatched_id1) + unmatched_id2)
                
             # Send row to database
            features = returned_json.get('matched').get('features')
            rows = []
            for feature in features:
                    if  feature['properties']['section'][0] > feature['properties']['section'][1]:
                        feature['properties']['section'][0] = 0
                        feature['properties']['section'][1] = 0

                    if  feature['properties']['referenceId'] is None:
                        null_matched.append((str(feature['properties']['{}'.format(primary_key1)]))+feature['properties']['{}'.format(primary_key2)])
                        continue

                    row = (feature['properties']['referenceId'], 
                               feature['properties']['fromIntersectionId'],
                               feature['properties']['toIntersectionId'],
                               feature['properties']['roadClass'], 
                               feature['properties']['direction'],
                               feature['properties']['referenceLength'],
                               feature['properties']['side'], 
                               feature['properties'].get('score', None),
                               '[' + str(feature['properties']['section'][0]) +', '+ str(feature['properties']['section'][1]) +')', 
                               'SRID=4326;LineString('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['coordinates'])+')', 
                               feature['properties']['originalFeature']['properties']['{}'.format(primary_key1)],
                               feature['properties']['originalFeature']['properties']['{}'.format(primary_key2)], 
                               query_num)


                    rows.append(row)



                    sql='INSERT INTO ' + output_table + '(reference_id, from_intersection, to_intersection, road_class, direction, reference_length, side,score, section, geometry, {}, {}, query_num) VALUES %s'.format(primary_key1, primary_key2)


                    with con:
                        with con.cursor() as cur:
                            execute_values(cur, 
                                       sql, 
                                       rows) 
            
    else:
        for i in range(0, count, 300):
        
            # First time calling function on input table         
            if previously_unmatched_id is None: 

                # two rows make up the primary key of the table
                get_centreline_sql =  '''SELECT 'Feature' as type,  json_build_object('{}', {}) as properties, ST_AsGeoJson(geom)::json as geometry 
                FROM {}
                LIMIT 300 OFFSET {}'''.format(primary_key1, str(primary_key1), input_table, i)


            # calling function again
            else:

                get_centreline_sql = '''SELECT 'Feature' as type,  json_build_object('{}', {}) as properties, ST_AsGeoJson(geom)::json as geometry 
                FROM {} WHERE {} IN {} 
                LIMIT 300 OFFSET {}'''.format(primary_key1, primary_key1, input_table, str(primary_key1), tuple(previously_unmatched_id), i)

            with con:
                with con.cursor(cursor_factory=RealDictCursor) as cur:
                    cur.execute(get_centreline_sql)
                    results = cur.fetchall()
                    final_json = json.dumps({"type": "FeatureCollection",
                                            "features": results})
            # Send data to SharedStreets API    
            base_url = "https://api.sharedstreets.io/v0.1.0/match/geoms"
            apiKey = "bdd23fa1-7ac5-4158-b354-22ec946bb575"
            params = {'ignoreDirection': 'false',
                      'bearingTolerance' : bearing_tolerance,
                      'searchRadius' : search_radius,
                      'auth' : apiKey,
                      'tileHierarchy': 6,
                      'dataSource': 'osm/planet-181029',
                      'lengthTolerance' : length_tolerance,
                      'snapTopology': 'true'
                     }
            headers= {"Content-type": "application/json; charset=UTF-8"}

            r = requests.post(base_url, params = params, headers = headers, json = {"type": "FeatureCollection", "features": results})
            returned_json = r.json()
            unmatched = returned_json.get('unmatched').get('features')
            for feature in unmatched:

                    unmatched_id1 = feature['properties']['{}'.format(primary_key1)]
                    unmatched_rows.append(str(unmatched_id1))
                
             # Send row to database
            features = returned_json.get('matched').get('features')
            rows = []
            for feature in features:
                    if  feature['properties']['section'][0] > feature['properties']['section'][1]:
                        feature['properties']['section'][0] = 0
                        feature['properties']['section'][1] = 0

                    if  feature['properties']['referenceId'] is None:
                        null_matched.append(str(feature['properties']['{}'.format(primary_key1)]))
                        continue

                    row = (feature['properties']['referenceId'], 
                               feature['properties']['fromIntersectionId'],
                               feature['properties']['toIntersectionId'],
                               feature['properties']['roadClass'], 
                               feature['properties']['direction'],
                               feature['properties']['referenceLength'],
                               feature['properties']['side'], 
                               feature['properties'].get('score', None),
                               '[' + str(feature['properties']['section'][0]) +', '+ str(feature['properties']['section'][1]) +')', 
                               'SRID=4326;LineString('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['coordinates'])+')', 
                               feature['properties']['originalFeature']['properties']['{}'.format(primary_key1)],
                               query_num)


                    rows.append(row)



                    sql='INSERT INTO ' + output_table + '(reference_id, from_intersection, to_intersection, road_class, direction, reference_length, side,score, section, geometry, {},  query_num) VALUES %s'.format(primary_key1)


                    with con:
                        with con.cursor() as cur:
                            execute_values(cur, 
                                       sql, 
                                       rows) 
    return unmatched_rows